In [2]:
import cv2
import os
import cv2
import os
import mahotas
import matplotlib.pyplot as plt
import numpy as np


cascPathface = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_frontalface_alt2.xml"
cascPatheyes = os.path.dirname(
    cv2.__file__) + "/data/haarcascade_eye_tree_eyeglasses.xml"

faceCascade = cv2.CascadeClassifier(cascPathface)
eyeCascade = cv2.CascadeClassifier(cascPatheyes)

video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(gray,
                                         scaleFactor=1.1,
                                         minNeighbors=5,
                                         minSize=(60, 60),
                                         flags=cv2.CASCADE_SCALE_IMAGE)
    for (x,y,w,h) in faces:
        
        c=25 #for putting object extended a bit out of the face boundaries
        
        faceROI = frame[y-c:y+h+c,x-c:x+w+c]
        fh, fw,_=faceROI.shape
       
        #image preprocessing according to the ROI
        img2 = cv2.imread('ppg.png')
        img2=cv2.resize(img2, (fw, fh),interpolation = cv2.INTER_AREA )
        img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        
        ret1, mask = cv2.threshold(img2gray, 229, 255, cv2.THRESH_BINARY_INV)
        mask_inv = cv2.bitwise_not(mask)

        img1_bg = cv2.bitwise_and(faceROI,faceROI,mask = mask_inv)
        img2_fg = cv2.bitwise_and(img2,img2,mask = mask)
        dst = cv2.add(img1_bg,img2_fg)
        
        frame[y-c:y+h+c,x-c:x+w+c] = dst

    # Display the resulting frame
    cv2.imshow('Face Video', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()